In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import io
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

2024-09-08 14:08:35.946370: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-08 14:08:35.975720: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-08 14:08:35.985109: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-08 14:08:36.034949: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-08 14:08:37.224967: W tensorflow/compiler/tf2

In [2]:
def generate_model_summary_image(model, training_params, performance_metrics, img_name):
    model_summary = []
    model.summary(print_fn=lambda x:model_summary.append(x))
    model_summary_str = "\n".join(model_summary)

    summary_text = (
        f"Model Summary:\n{model_summary_str}\n\n"
        f"Training Parameters:\n"
        f"Epochs: {training_params['epochs']}, Batch Size: {training_params['batch_size']}\n\n"
        f"Performance Metrics:\n"
        f"Test Accuracy: {performance_metrics['accuracy']:.2f}%, Test Loss: {performance_metrics['loss']:.4f}"
    )

    width, height = 800, 600
    img = Image.new('RGB', (width, height), color=(255, 255, 255))
    draw = ImageDraw.Draw(img)

    try:
        font = ImageFont.truetype("DejaVuSans-Bold.ttf", 14)
    except IOError:
        font = ImageFont.load_default()

    padding = 10
    draw.text((padding, padding), summary_text, fill=(0, 0, 0), font=font)

    img.save(img_name)
    print(f'Model summary saved as {img_name}.')



In [6]:
#Dataset Imported
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

print(f"Dimensions of training features: {X_train.shape}")
print(f"Dimensions of training labels: {y_train.shape}")
print(f"Dimensions of test features: {X_test.shape}")
print(f"Dimensions of test labels: {y_test.shape}")

Dimensions of training features: (60000, 28, 28)
Dimensions of training labels: (60000,)
Dimensions of test features: (10000, 28, 28)
Dimensions of test labels: (10000,)


In [7]:
#Reshape to original 28 x 28
training_features = X_train.reshape((-1, 28, 28, 1))

#Normalizing the pixel values
training_features = training_features / 255.0

print(f"New dimensions: {training_features.shape}")

New dimensions: (60000, 28, 28, 1)


In [6]:
#Basic model
baseline_model = tf.keras.models.Sequential(
    layers = [
        tf.keras.layers.Input(shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(filters=5, kernel_size=(3, 3), strides=(1, 1), padding='same'),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=10, activation='softmax')
    ],
    name = "Baseline_Model"
)
baseline_model.summary()

I0000 00:00:1725784610.565362     692 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725784610.808051     692 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725784610.808095     692 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725784610.810679     692 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725784610.810724     692 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "Baseline_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 5)      │            50 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 980)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         9,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,860 (38.52 KB)

 Trainable params: 9,860 (38.52 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
baseline_model.compile(optimizer='adam' , loss='sparse_categorical_crossentropy', metrics=['accuracy'])
baseline_model.fit(training_features, y_train, epochs=30, batch_size=32)

Epoch 1/30


I0000 00:00:1725784612.154058   16709 service.cc:146] XLA service 0x7f89b0008a90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725784612.154102   16709 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-09-08 13:36:52.206856: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-08 13:36:52.350922: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


  83/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4195 - loss: 1.7126

I0000 00:00:1725784614.103046   16709 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.7613 - loss: 0.7020
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8623 - loss: 0.4012
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8753 - loss: 0.3622
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8814 - loss: 0.3399
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8832 - loss: 0.3303
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8870 - loss: 0.3202
Epoch 7/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8911 - loss: 0.3118
Epoch 8/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8940 - loss: 0.3003
Epoch 9/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8938 - loss: 0.3003
Epoch 10/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8949 - loss: 0.2937
Epoch 11/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8960 - loss: 0.2901
Epoch 12/30
1875/1875 ━━━━━━━━━━━━━━━━━━━

In [8]:
test_loss, test_accuracy = baseline_model.evaluate(X_test, y_test)
train_loss, train_accuracy = baseline_model.evaluate(training_features, y_train)

print(f"Training accuracy: {train_accuracy:.2f}")
print(f"Training loss: {train_loss:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")
print(f"Test loss: {test_loss:.2f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8903 - loss: 0.3328
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9127 - loss: 0.2442
Training accuracy: 0.91
Training loss: 0.25
Test accuracy: 0.89
Test loss: 0.33


In [9]:
lenet = Sequential([
    Input(shape=(28, 28, 1)),
    Conv2D(filters=6, kernel_size=(5, 5), strides=(1, 1), padding='same'),
    MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),
    Conv2D(filters=16, kernel_size=(5, 5)),
    MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    Flatten(),
    Dense(units=120, activation='relu'),
    Dense(units=84, activation='relu'),
    Dense(units=10, activation='softmax')
], 
name='lenet')
lenet.summary()

Model: "lenet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 6)      │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 6)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 10, 10, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 5, 5, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 120)            │        48,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 61,706 (241.04 KB)

 Trainable params: 61,706 (241.04 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
lenet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
lenet.fit(training_features, y_train, epochs=15, batch_size=32)

Epoch 1/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.7488 - loss: 0.6854
Epoch 2/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8755 - loss: 0.3442
Epoch 3/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8915 - loss: 0.2964
Epoch 4/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9031 - loss: 0.2594
Epoch 5/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9119 - loss: 0.2384
Epoch 6/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9193 - loss: 0.2182
Epoch 7/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9256 - loss: 0.2002
Epoch 8/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9328 - loss: 0.1798
Epoch 9/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9375 - loss: 0.1660
Epoch 10/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9422 - loss: 0.1558
Epoch 11/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9461 - loss: 0.1397
Epoch 12/15
1875/1875 ━━━━━━━━

In [11]:
test_loss, test_accuracy = lenet.evaluate(X_test, y_test)
train_loss, train_accuracy = lenet.evaluate(training_features, y_train)

print(f"Training accuracy: {train_accuracy:.2f}")
print(f"Training loss: {train_loss:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")
print(f"Test loss: {test_loss:.2f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8983 - loss: 0.3852
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9685 - loss: 0.0835
Training accuracy: 0.97
Training loss: 0.08
Test accuracy: 0.90
Test loss: 0.37


# Hyperparameter tuning

In [12]:
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input

def build_model(hp):
    model = keras.Sequential()
    model.add(Input(shape=(28, 28, 1)))

    model.add(
        Conv2D(
            filters=hp.Int('filters_1', min_value=16, max_value=64, step=16),
            kernel_size=hp.Choice('kernel_size_1', values=[3, 5]),
            activation='relu',
            padding='same'
        )
    )
    model.add(MaxPooling2D(pool_size=(2, 2), padding='valid'))
    model.add(Flatten())
    model.add(
        Dense(
            units=hp.Int('unts', min_value = 64, max_value=256, step=64),
            activation='relu'
        )
    )
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(
        hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    ),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
    )
    return model

In [13]:
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20, 
    factor=3,
    directory='hyperparam_search',
    project_name='fashoin_mnist_tuning'
)

Reloading Tuner from hyperparam_search/fashoin_mnist_tuning/tuner0.json


In [14]:
#Early stopping
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(training_features, y_train, 20, validation_data=(X_test, y_test), callbacks=[stop_early])

In [15]:
best_model = tuner.get_best_models(num_models=1)[0]

test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
train_loss, train_accuracy = best_model.evaluate(training_features, y_train)

print(f"Training accuracy: {train_accuracy:.2f}")
print(f"Training loss: {train_loss:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")
print(f"Test loss: {test_loss:.2f}")

/home/wiki/.local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9234 - loss: 0.2183
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9556 - loss: 0.1298
Training accuracy: 0.95
Training loss: 0.13
Test accuracy: 0.93
Test loss: 0.22


In [19]:
best_model.save(filepath='fashion_mnist_model.h5')


In [20]:
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 48)     │         1,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 48)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       602,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 604,074 (2.30 MB)

 Trainable params: 604,074 (2.30 MB)

 Non-trainable params: 0 (0.00 B)